In [1]:
data = [(['Linux', 'is', 'the', 'best', 'OS'], ['OS','IR','IR','IR','IR']),
(['Ubuntu', 'is', 'my', 'favourite', 'OS'], ['OS','IR','IR','IR','IR'])]
corpus = []
for (doc, tags) in data:
    doc_tag = []
    for word, tag in zip(doc,tags):
        doc_tag.append((word, tag))
    corpus.append(doc_tag)
print(corpus)

[[('Linux', 'OS'), ('is', 'IR'), ('the', 'IR'), ('best', 'IR'), ('OS', 'IR')], [('Ubuntu', 'OS'), ('is', 'IR'), ('my', 'IR'), ('favourite', 'IR'), ('OS', 'IR')]]


In [4]:
##CRF  for NER
def doc2features(doc, i):
    word = doc[i:][0]
   
    
    # Features from current word
    features={
        'word.word': word,
    }
    # Features from previous word
    if i > 0:
        prevword = doc[i-1:][0]
        
        features['word.prevword'] = prevword
    else:
        features['BOS'] = True # Special "Beginning of Sequence" tag
        
    # Features from next word
    if i < len(doc)-1:
        nextword = doc[i+1:][0]
        features['word.nextword'] = nextword
    else:
        features['EOS'] = True # Special "End of Sequence" tag
    return features
 
def extract_features(docs):
    return [doc2features(doc, i) for i in range(len(doc))]
 
X = [extract_features(doc) for doc in corpus]
print(X)

##The features can be modified based on our need, we can include 2 previous 2 next or more window

[[{'word.word': 'Ubuntu', 'BOS': True, 'word.nextword': 'is'}, {'word.word': 'is', 'word.prevword': 'Ubuntu', 'word.nextword': 'my'}, {'word.word': 'my', 'word.prevword': 'is', 'word.nextword': 'favourite'}, {'word.word': 'favourite', 'word.prevword': 'my', 'word.nextword': 'OS'}, {'word.word': 'OS', 'word.prevword': 'favourite', 'EOS': True}], [{'word.word': 'Ubuntu', 'BOS': True, 'word.nextword': 'is'}, {'word.word': 'is', 'word.prevword': 'Ubuntu', 'word.nextword': 'my'}, {'word.word': 'my', 'word.prevword': 'is', 'word.nextword': 'favourite'}, {'word.word': 'favourite', 'word.prevword': 'my', 'word.nextword': 'OS'}, {'word.word': 'OS', 'word.prevword': 'favourite', 'EOS': True}]]


In [5]:
##Break the tuple to extract the labels which will be our Y
def get_labels(doc):
    return [tag for (token,tag) in doc]
y = [get_labels(doc) for doc in corpus]
print(y)

[['OS', 'IR', 'IR', 'IR', 'IR'], ['OS', 'IR', 'IR', 'IR', 'IR']]


In [6]:
import sklearn_crfsuite
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=20,
    all_possible_transitions=False,
)
crf.fit(X, y);

In [8]:
test = [['CentOS', 'is', 'my', 'favourite', 'OS']]
X_test = extract_features(test)
print(crf.predict_single(X_test))


['OS', 'IR', 'IR', 'IR', 'IR']


In [ ]:
##For chunking like Names we might have to do POS tagging as well in features